In [0]:
DECLARE OR REPLACE VARIABLE var_recent_ts TIMESTAMP;
SET VAR var_recent_ts =
(SELECT MAX(load_timestamp) recent_ts
FROM default.sales_brnz);

In [0]:
CREATE OR REPLACE TEMP VIEW sales_fact_temp_v AS
SELECT c.id as country_id, i.id as item_type_id, o.id as order_type_id,
b.order_date,b.order_id,b.ship_date,b.units_sold,b.unit_price,b.unit_cost,b.total_revenue,b.total_cost,b.total_profit,CURRENT_TIMESTAMP() as load_timestamp, CURRENT_USER() as loaded_by
FROM sales_brnz b
JOIN country_dim_slvr c 
ON c.country = b.country
JOIN item_type_dim_slvr i 
ON i.item_type = b.item_type
JOIN order_type_dim_slvr o  
ON o.sales_channel = b.sales_channel AND o.order_priority = b.order_priority
WHERE load_timestamp = var_recent_ts 

In [0]:
MERGE INTO sales_fact_slvr t
USING sales_fact_temp_v s
ON t.order_id = s.order_id
WHEN MATCHED THEN UPDATE SET t.load_timestamp = s.load_timestamp, t.loaded_by = s.loaded_by
WHEN NOT MATCHED THEN INSERT *